In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures, StandardScaler
from sklearn import tree,linear_model,neighbors
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":10,"axes.titlesize":24,"axes.labelsize":24}) 
from sklearn.linear_model import Lasso

# Upload data

In [2]:
train = pd.read_csv('TrainClean.csv',index_col=0).reset_index(drop=True)
test = pd.read_csv('TestClean.csv',index_col=0).reset_index(drop=True)
train.describe()

,IsHoliday,HasPromotions,NearestCompetitor,Region_AreaKM2,Region_GDP,Region_PopulationK,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,...,Fog,Thunderstorm,Hail,Nothing,Rain,ZeroClouds,H,WCI,NumberOfSales,NumberOfCustomers
count,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.00000,...,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000
mean,0.000945,0.449507,8042.993882,12313.341916,14045.556837,3845.765475,6.310558,74.924271,1015.113640,10.33585,...,0.254421,0.061453,0.006415,0.238873,0.592698,0.000129,10.336109,561.257762,8.401363,312.547433
std,0.030723,0.497444,11640.203554,9114.075588,2760.416990,2816.324768,5.797125,12.764557,7.728213,7.04847,...,0.435536,0.240160,0.079839,0.426396,0.491333,0.011359,7.048560,190.146041,0.440376,158.106619
min,0.000000,0.000000,47.000000,344.000000,9893.000000,816.000000,-7.000000,46.000000,999.000000,-13.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.999909,83.582039,0.000000,0.000000
25%,0.000000,0.000000,1057.000000,7215.000000,11849.000000,1293.000000,2.000000,66.000000,1010.000000,4.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000191,413.122279,8.127700,214.000000
50%,0.000000,0.000000,3307.000000,9337.000000,15017.000000,1892.000000,7.000000,76.000000,1015.000000,11.00000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,11.000220,552.958918,8.405256,278.000000
75%,0.000000,1.000000,9761.000000,15566.000000,15931.000000,5727.000000,11.000000,85.000000,1020.000000,16.00000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,16.000332,707.277778,8.684739,369.000000
max,1.000000,1.000000,85070.000000,32221.000000,23931.000000,8146.000000,20.000000,100.000000,1032.000000,30.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000379,1268.803656,10.190244,2206.000000


In [3]:
train.columns

Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_GDP', 'Region_PopulationK', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Precipitationmm', 'StandardMarket',
       'HyperMarket', 'SuperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'WithNonFoodDepartment', 'Region0', 'Region1',
       'Region2', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'Region9', 'January', 'February', 'March', 'April', 'May',
       'June', 'July', 'August', 'September', 'October', 'November',
       'December', 'Tuesday', 'Wednesday', 'Friday', 'Saturday', 'Monday',
       'Thursday', 'Sunday', 'Snow', 'Fog', 'Thunderstorm', 'Hail', 'Nothing',
       'Rain', 'ZeroClouds', 'H', 'WCI', 'NumberOfSales', 'NumberOfCustomers'],
      dtype='object')

# Normalization

In [4]:
all_data_index = train.columns.drop(['NumberOfSales','NumberOfCustomers'])
all_data = pd.concat((train.loc[:,all_data_index],test.loc[:,all_data_index]), ignore_index=True)
columns = all_data.columns

In [5]:
x = all_data.values
min_max_scaler = MinMaxScaler()
all_data_normalized = pd.DataFrame(min_max_scaler.fit_transform(x))
all_data_normalized.columns = columns

In [6]:
test_offset = train.shape[0]
train_normalized = all_data_normalized.iloc[0:test_offset,:]
train_normalized['NumberOfSales'] = train["NumberOfSales"]
train_normalized['NumberOfCustomers'] = train["NumberOfCustomers"]

In [7]:
test_normalized = all_data_normalized.iloc[test_offset:all_data_normalized.shape[0],:]
train=train_normalized
test=test_normalized

# Lasso for Number Of Customers

In [8]:
y=train['NumberOfCustomers'].values.reshape(-1,1)
x=train.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)

In [9]:
NumOfCustF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    
    if abs(c)>threshold and not i == 0:
        NumOfCustF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfCustF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.332
	                  Train RMSE=16694.844

Linear Regression with Lasso - Chosen Features
HasPromotions	56.247
NearestCompetitor	-102.879
Region_AreaKM2	-53.353
Mean_Dew_PointC	-10.855
Mean_Sea_Level_PressurehPa	6.986
Precipitationmm	-23.298
StandardMarket	-66.955
HyperMarket	-11.409
ShoppingCenter	292.587
General	-25.885
WithFishDepartment	373.462
Region0	6.260
Region1	-18.470
Region3	50.441
Region4	88.570
Region5	-18.353
Region6	-28.471
Region7	-24.489
Region8	-57.468
Region9	9.854
January	-10.239
February	-7.361
April	4.948
May	9.440
June	5.140
September	-4.966
October	-4.173
November	0.627
December	50.670
Wednesday	-11.938
Friday	4.912
Saturday	-18.015
Monday	34.161
Thursday	-7.040
Snow	-5.572
Fog	-0.630


Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'Region0', 'Region1', 'Region3', 'Region4',
       'Region5', 'Region6', 'Region7', 'Region8', 'Region9', 'January',
       'February', 'April', 'May', 'June', 'September', 'October', 'November',
       'December', 'Wednesday', 'Friday', 'Saturday', 'Monday', 'Thursday',
       'Snow', 'Fog'],
      dtype='object')

# Lasso for number of sales with direct passage

In [10]:
y=train['NumberOfSales'].values.reshape(-1,1)
x=train.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)

In [11]:
NumOfCustF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    
    if abs(c)>threshold and not i == 0:
        NumOfCustF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfCustF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.283
	                  Train RMSE=0.139

Linear Regression with Lasso - Chosen Features
HasPromotions	0.331
NearestCompetitor	-0.025
Region_GDP	0.055
Mean_Sea_Level_PressurehPa	0.039
Mean_TemperatureC	-0.033
Precipitationmm	-0.046
StandardMarket	-0.004
HyperMarket	-0.031
SuperMarket	0.000
ShoppingCenter	0.319
General	-0.158
WithFishDepartment	0.009
Region0	0.054
Region2	0.040
Region3	0.227
Region4	0.110
Region5	-0.031
Region6	-0.073
Region8	-0.097
Region9	0.080
January	-0.019
February	-0.008
March	-0.002
April	0.001
May	0.023
June	0.014
September	-0.032
October	-0.022
November	0.022
December	0.224
Tuesday	0.040
Wednesday	-0.000
Friday	0.059
Saturday	-0.014
Monday	0.174
Snow	-0.007
Fog	-0.001
Rain	0.003


Index(['HasPromotions', 'NearestCompetitor', 'Region_GDP',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'Region0', 'Region2', 'Region3',
       'Region4', 'Region5', 'Region6', 'Region8', 'Region9', 'January',
       'February', 'March', 'April', 'May', 'June', 'September', 'October',
       'November', 'December', 'Tuesday', 'Wednesday', 'Friday', 'Saturday',
       'Monday', 'Snow', 'Fog', 'Rain'],
      dtype='object')

# Normalization of number of customers

In [12]:
x = train['NumberOfCustomers'].values
min_max_scaler = MinMaxScaler()
train['NumberOfCustomers'] = min_max_scaler.fit_transform(x.reshape(-1,1))
x = train.drop(columns=['NumberOfSales'],axis=1)
y =train['NumberOfSales'].values.reshape(-1,1)

# Lasso for Number Of Sales

In [13]:
NumOfSalesF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    #print("%d\t%3.3f" % (i,c) )
    if abs(c)>threshold and not i == 0:
        NumOfSalesF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfSalesF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.770
	                  Train RMSE=0.045

Linear Regression with Lasso - Chosen Features
IsHoliday	-0.067
HasPromotions	0.199
NearestCompetitor	0.202
Region_AreaKM2	0.146
Region_PopulationK	0.069
Mean_Humidity	0.019
Mean_Sea_Level_PressurehPa	0.023
Mean_TemperatureC	-0.054
Mean_VisibilityKm	-0.001
Precipitationmm	-0.009
StandardMarket	0.159
HyperMarket	-0.000
SuperMarket	0.005
ShoppingCenter	-0.350
General	-0.004
WithFishDepartment	-0.684
WithNonFoodDepartment	0.094
Region0	0.031
Region1	0.028
Region3	0.116
Region4	-0.027
Region5	0.002
Region6	-0.035
Region7	0.059
Region8	0.023
January	-0.015
February	-0.012
March	-0.020
April	-0.012
May	0.000
June	0.007
July	0.011
August	0.000
September	-0.024
October	-0.021
November	0.008
December	0.091
Tuesday	0.015
Wednesday	-0.000
Friday	0.022
Monday	0.068
Thursday	-0.008
Sunday	-0.167
Snow	-0.005
Fog	-0.005
Nothing	-0.002
Rain	0.004
NumberOfCustomers	5.202


Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'WithNonFoodDepartment', 'Region0',
       'Region1', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December',
       'Tuesday', 'Wednesday', 'Friday', 'Monday', 'Thursday', 'Sunday',
       'Snow', 'Fog', 'Nothing', 'Rain', 'NumberOfCustomers'],
      dtype='object')

In [14]:
len(NumOfSalesF)

48

In [15]:
len(NumOfCustF)

38